In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map 

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# ee.Reducer.mode()的用法

In [ ]:
lt = ee.List([0,0,0,0,0,1,1,1,1,2,2])
mode = ee.Reducer.mode(2,1)
lt_mode = lt.reduce(mode)
print(lt_mode.getInfo())

# 查看图像的投影和像元大小

In [ ]:
# projection and 
image = ee.Image('MODIS/MOD13A1/MOD13A1_005_2014_05_09')
visParams = {'min': 0.15, 'max': 0.7} 
Map.setCenter(-77.5127,43.2642,11) 
Map.addLayer(image, visParams, 'original')

print(image.bandNames().getInfo())
print('Projection and transformation information:', image.projection().getInfo()['crs'])
print('Pixel size in meters:', image.projection().nominalScale().getInfo())

reprojected = image.reproject('EPSG:4326')
Map.addLayer(reprojected, visParams, 'Reprojected')

# 图像校正

In [ ]:
# registerion
image1 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150502T082736Z')
image2 = ee.Image('SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL/s01_20150305T081019Z')

image1Orig = image1.resample('bicubic')
image2Orig = image2.resample('bicubic')
 
image1RedBand = image1Orig.select('R')
image2RedBand = image2Orig.select('R')
 
displacement = image2RedBand.displacement(**{
    'referenceImage': image1RedBand,  
    'maxOffset': 50.0,      
    'patchWidth': 100.0   
})  
registered = image2Orig.displace(displacement)
 
visParam = {'bands': ['R', 'G', 'B'], 'max': 4000}
Map.addLayer(image1Orig, visParam, 'Reference'); 
Map.addLayer(image2Orig, visParam, 'Before Registration') 
Map.addLayer(registered, visParam, 'After Registration')

# 计算熵

In [ ]:
image = ee.Image('USDA/NAIP/DOQQ/m_4307663_se_18_1_20130619')

Map.setCenter(-76.13, 43.03,16)
Map.addLayer(image, {'max': 255}, 'Input image')

nir = image.select('N')
square = ee.Kernel.square(**{'radius': 3})
entropy = nir.entropy(square)

Map.addLayer(entropy,{'min': 1, 'max': 5, 'palette': ['0000CC', 'CC0000']}, 'entropy')

# 直方图
histogram = entropy.select('N').reduceRegion(**{
    'reducer': ee.Reducer.histogram(100),
    'geometry': image.geometry(), 
    'scale': 100,
    'bestEffort': True
})
print(histogram.getInfo())

# 图像融合

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_121038_20160115')
Map.addLayer(image,{'bands': ['B4', 'B3', 'B2'],'min':0,'max':3000}, 'RGB image')
hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()

combined = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), image_toa.select('B8')])

sharpened = ee.Image(combined).hsvToRgb()
Map.centerObject(image,8)
Map.addLayer(sharpened,'','pan-sharpened')

# 边缘检测

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_015030_20140307').select("B8")
Map.setCenter(-76.1467, 43.0458,12)
Map.addLayer(image, '', 'input image')

canny = ee.Algorithms.CannyEdgeDetector(**{'image': image, 'threshold': 200, 'sigma': 1 })
Map.addLayer(canny, {}, 'canny')

hough = ee.Algorithms.HoughTransform(canny, 256, 7000, 80)
Map.addLayer(hough, {}, 'hough')

# 定义无geometry的要素

In [ ]:
ab = ee.Feature(None,{'time': 0,'area': 1,'area_smoothed': 2})
print(ab.getInfo())

# 提取字典的属性

In [ ]:
print(ee.Date('2020-01-01').getInfo())
print(ee.Date('2020-01-01').getInfo()['value'])

# 定义一个要素，没有几何属性

In [ ]:
a = ee.Feature(None, {'foo': 1})
print(a.getInfo())

# 移除要素的某个属性

In [ ]:
# feat为输入要输
def removeProperty(feat):
    selectProperties = ee.Feature(feat).propertyNames().filter(ee.Filter.neq('item', 'waterclass'))
    return feat.select(selectProperties)

# 多个列表的配对

In [4]:
list1 = ee.List(['20190101T160509_20190101T160509_T17RNJ',
                 '20190104T160649_20190104T161509_T17RLH',
                 '20190104T160649_20190104T161509_T17RMJ',
                 '20190104T160649_20190104T161509_T17RMK'])
list2 = ee.List(['17RNJ','17RLH','17RMJ','17RMK'])
list3 = ee.List([1,2,3,4])

In [6]:
a = ee.List.sequence(0,list1.length().subtract(1),1)
paired = a.map(lambda i : [list1.get(i),list2.get(i),list3.get(i)])
print(paired.getInfo())

[['20190101T160509_20190101T160509_T17RNJ', '17RNJ', 1], ['20190104T160649_20190104T161509_T17RLH', '17RLH', 2], ['20190104T160649_20190104T161509_T17RMJ', '17RMJ', 3], ['20190104T160649_20190104T161509_T17RMK', '17RMK', 4]]
